MAC 5768 - Visão e Processamento de Imagens

Segundo Semestre - 2020

Resumo de Projeto

Equipe:

    Ciro B Rosa - ciro.rosa@alumni.usp.br
    Josilton Sousa - josilton.sousa@gmail.com

Projeto de elaboração de tarefas do curso de Visão e Processamento de Imagens.


Fase 2 - Parte 1 - Entrega em 9/11/2020

Objetivos:
* A partir de 1080 fotos coloridas de 10 categorias de objetos (originalDataset), criar duplicatas em escala de cinza e armazená-las na pasta originalGrayDataset.
* A partir das imagens em escala cinza, obter um dataset aumentado e armazenar em augmentedDataset.

As funções para geração do augmentedDataset são:
* Escala de cinza
* Soma do fundo da imagem (gradiente) com a imagem cinza
* Log2 da imagem cinza
* Função Gama (exponencial da imagem)
* Média e convolução, com o uso de janela uniforme 3x3.


Código para geração do originalGrayDataset a partir do originalDataset

Este código terá as seguintes funções:
* Leitura dos metadados originais
* Geração de imagens em escala cinza
* Criação de metadados específicos para as imagens geradas
* Gravação de dados em originalGrayDataset

In [1]:
#import numpy as np
#from scipy.signal import convolve2d as conv2
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
import pandas as pd

# arquivo de metadados em cores
pasta1 = "./originalDataset/"
metafile1 = "grade-fotos.csv"
filename1 = pasta1 + metafile1
df = pd.read_csv(filename1, sep=";")
print(df.head(5), "\n")

# arquivo de metadados em cinza
pasta2 = "./originalGrayDataset/"
metafile2 = "grade-cinza.csv"
filename2 = pasta2 + metafile2

# adequação do nome de arquivo nos metadados
def fix_jpeg_name(arq):
    arq2 = int(arq[:-4])
    arq2 = str(arq2) + ".jpg"
    return arq2
    

# quantidade de fotos
n = len(df.arquivo)

# criar dataframe vazio para metadados em cinza
df2 = pd.DataFrame(data=None, columns=df.columns)
df2.insert(loc=5, column="transformacao", value=[])
df2_length = len(df2)

# transformação de fotos cinza
for ind in df.index:
    #print(df.loc[ind])

    # ler foto colorida para a lista
    arq1 = df["arquivo"][ind]
    arq1 = fix_jpeg_name(arq1)
    f1 = pasta1 + arq1
    img = io.imread(f1)
    
    # gerar foto cinza
    img_gray = rgb2gray(img)
    img_gray = img_as_ubyte(img_gray)   # voltar para níveis 0-255
     
    # nome do arquivo da foto cinza
    arq1 = int(arq1[:-4])
    arq2 = arq1 + n
    arq2 = str(arq2) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto cinza
    io.imsave(f, img_gray)
     
    # gravar dados da foto cinza no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "cinza", df.responsavel[ind], arq2]
    df2.loc[df2_length] = l
    df2_length += 1

# salvar metadados em cinza
print(df2.head(5), "\n")
df2.to_csv(filename2, index = False, header=True, sep=";")


   sequencia objeto tipo_obj   fundo  iluminacao responsavel arquivo
0          1  garfo        a  branco  indoor dia    Josilton  1 .jpg
1          2  garfo        a  branco  indoor dia    Josilton  2 .jpg
2          3  garfo        a  branco  indoor dia    Josilton  3 .jpg
3          1   faca        a  branco  indoor dia    Josilton  4 .jpg
4          2   faca        a  branco  indoor dia    Josilton  5 .jpg 

  sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0         1  garfo        a  branco  indoor dia         cinza    Josilton   
1         2  garfo        a  branco  indoor dia         cinza    Josilton   
2         3  garfo        a  branco  indoor dia         cinza    Josilton   
3         1   faca        a  branco  indoor dia         cinza    Josilton   
4         2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   



Código para geração do augmentedDataset a partir do originalGrayDataset

Inicialmente, a base de dados e metadados são copiados da base cinza para a base augmented. Em seguida, as funções de transformação são executadas em sequência para cada foto.

In [3]:
import numpy as np
from scipy.signal import convolve2d as conv2
from skimage import io
#from skimage.color import rgb2gray
#from skimage.util import img_as_ubyte
import pandas as pd


# definição de funções de transformação
def transf_grad(img):
    img_aux = np.gradient(img)
    img2 = np.sqrt(np.square(img_aux[0]) + np.square(img_aux[1]))
    return img2

def transf_log2(img, c):
    img2 = c * np.log2(1.0 + img)
    return img2

def transf_gama(img, c, gama):
    img2 = c * np.power(img, gama)
    return img2

# adequação do nome de arquivo nos metadados
def fix_jpeg_name(arq):
    arq2 = int(arq[:-4])
    arq2 = str(arq2) + ".jpg"
    return arq2


# arquivo de metadados em cinza
pasta1 = "./originalGrayDataset/"
metafile1 = "grade-cinza.csv"
filename1 = pasta1 + metafile1
df = pd.read_csv(filename1, sep=";")
print(df.head(5), "\n")

# arquivo de metadados aumentados
pasta2 = "./augmentedDataset/"
metafile2 = "grade-augmented.csv"
filename2 = pasta2 + metafile2

# criar dataframe para metadados aumentados
df2 = pd.DataFrame(data=df, index=df.index, columns=df.columns)

# número do último arquivo
n = df2.tail(1)
n = n["arquivo"]
n = n.values[0]
n = str(n[:-4])
n = int(n)


# transformação de fotos cinza
for ind in df.index:
    #print(df.loc[ind])

    # ler foto cinza
    arq1 = df["arquivo"][ind]
    arq1 = fix_jpeg_name(arq1)
    f1 = pasta1 + arq1
    img_gray = io.imread(f1)
    
    
    ### soma de fundo (gradiente) com foto cinza
    ###
    img_grad = transf_grad(img_gray)
    img_grad = img_gray + img_grad
    img_grad = img_grad.astype(np.uint8)
     
    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_grad)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "grad", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
    ### foto log2
    ###
    img_log2 = transf_log2(img_gray, c = 1)
    #img_log2 = img_log2.astype(np.uint8)
    
    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_log2)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "log", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
    ### foto exponencial
    ###
    img_gama = transf_gama(img_gray, c = 1, gama = 2)

    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_gama)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "exp", df.responsavel[ind], arq2]
    df2.loc[n] = l

    ### média e convolução
    ###
    psf = np.ones((3, 3)) / 9
    img_conv = conv2(img_gray, psf, 'same')
    img_conv = img_conv.astype(np.uint8)

    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_conv)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "conv", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
# salvar metadados aumentados
print(df2.head(5), "\n")
df2.to_csv(filename2, index = False, header=True, sep=";")


   sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0          1  garfo        a  branco  indoor dia         cinza    Josilton   
1          2  garfo        a  branco  indoor dia         cinza    Josilton   
2          3  garfo        a  branco  indoor dia         cinza    Josilton   
3          1   faca        a  branco  indoor dia         cinza    Josilton   
4          2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   



Lossy conversion from float64 to uint8. Range [5.285402218862249, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.523561956057013, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.321928094887362, 7.813781191217037]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.169925001442312, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [2.807354922057604, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [2.807354

Lossy conversion from float64 to uint8. Range [4.321928094887363, 7.876516946564999]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.129283016944966, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.584962500721156, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.247927513443585, 7.954196310386875]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.357552004618084, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.20945336562895, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.930737337562887, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversi

Lossy conversion from float64 to uint8. Range [4.523561956057013, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.285402218862249, 7.971543553950772]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.954196310386875, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.584962500721156, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.459431618637297, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.321928094887363, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.554588851677638, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float

Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [2.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.700439718141092, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.807354922057604, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.169925001442312, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.087462841250339, 8.0]. Convert image to uint8 pr

...

Lossy conversion from float64 to uint8. Range [1.584962500721156, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [2.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.4594316186372973, 7.554588851677638]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.169925001442312, 7.599912842187128]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.0, 7.515699838284043]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.

Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [2.321928094887362, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.6865005271832185]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [1.0, 7.679480099505446]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.700439718141092]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.807354922057604, 8.0]. Convert im

Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [1.584962500721156, 7.9886846867721655]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.392317422778761, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.807354922057604, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.754887502163468, 7.93073733756288

Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.906890595608519]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.857980995127572]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.845490050944375]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 8.0]. Convert image to uint8 prior to saving 

   sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0          1  garfo        a  branco  indoor dia         cinza    Josilton   
1          2  garfo        a  branco  indoor dia         cinza    Josilton   
2          3  garfo        a  branco  indoor dia         cinza    Josilton   
3          1   faca        a  branco  indoor dia         cinza    Josilton   
4          2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   

